# 🧠 Introducción a la Clasificación de Imágenes Médicas con Redes Neuronales

En este notebook, aprenderemos cómo utilizar redes neuronales convolucionales (CNN) para clasificar imágenes médicas en diferentes categorías. Utilizaremos el conjunto de datos Medical MNIST, que es ideal para principiantes debido a su tamaño reducido y formato estandarizado.

📁 Estructura del Dataset

El dataset está organizado en carpetas, una por cada clase:

MNIST/
├── AbdominalCT/
├── BrainMRI/
├── BreastMRI/
├── ChestCT/
├── ChestXRay/
└── Hand/

Cada carpeta contiene imágenes en formato .jpeg correspondientes a su categoría.

## 🔧 Preparación del Entorno

Antes de comenzar, asegúrate de tener instaladas las siguientes bibliotecas:

In [5]:
!pip3 install --user tensorflow
!pip3 install matplotlib
!pip3 install scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/fb/59/3eb58629e3749d9f4fc1e522487af369f9bd4c451f465d3054961fab6bf8/tensorflow-2.13.1-cp38-cp38-win_amd64.whl.metadata
  Using cached tensorflow-2.13.1-cp38-cp38-win_amd64.whl.metadata (2.6 kB)
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/5b/6f/8b125d126d54061c0be610b135aaf2f8960f55c2e185ac32445e4a5012d5/tensorflow-2.13.0-cp38-cp38-win_amd64.whl.metadata
  Using cached tensorflow-2.13.0-cp38-cp38-win_amd64.whl.metadata (2.6 kB)
  Obtaining dependency information for tensorflow-intel==2.13.0 from https://files.pythonhosted.org/packages/38/ba/dd4d998a852451e98dc009ecb208bbb0eeb0c8252dc35b7c4e1050762b36/tensorflow_intel-2.13.0-cp38-cp38-win_amd64.whl.metadata
  Using cached tensorflow_intel-2.13.0-cp38


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 📥 Carga y Preprocesamiento de los Datos

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

ImportError: initialization failed

In [ ]:
# Definir rutas
dataset_path = 'MNIST'  # Reemplaza con la ruta a tu dataset

# Parámetros
img_height, img_width = 64, 64
batch_size = 32

In [ ]:
# Generadores de datos para entrenamiento y validación
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

## 🧠 Construcción del Modelo de Red Neuronal Convolucional

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:

model.summary()

## 🏋️ Entrenamiento del Modelo

In [ ]:
epochs = 10

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

## 📈 Evaluación del Modelo

In [ ]:
# Evaluar el modelo en el conjunto de validación
loss, accuracy = model.evaluate(validation_generator)
print(f'Precisión en el conjunto de validación: {accuracy*100:.2f}%')

## 📊 Visualización de Resultados

In [ ]:
# Graficar precisión y pérdida durante el entrenamiento
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Precisión de entrenamiento')
plt.plot(epochs_range, val_acc, label='Precisión de validación')
plt.legend(loc='lower right')
plt.title('Precisión durante el entrenamiento')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Pérdida de entrenamiento')
plt.plot(epochs_range, val_loss, label='Pérdida de validación')
plt.legend(loc='upper right')
plt.title('Pérdida durante el entrenamiento')

plt.show()

## 🖼️ Prueba con Nuevas Imágenes

In [ ]:
from tensorflow.keras.preprocessing import image

# Cargar y preprocesar una imagen
img_path = 'ruta/a/una/imagen.jpg'  # Reemplaza con la ruta a tu imagen
img = image.load_img(img_path, target_size=(img_height, img_width), color_mode='grayscale')
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Realizar predicción
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction[0])
class_labels = list(train_generator.class_indices.keys())
print(f'La imagen pertenece a la clase: {class_labels[predicted_class]}')